## Packages needed

In [5]:
import geopandas as gpd
import os
import fiona
import rasterio.mask
from rasterio.fill import fillnodata
from rasterstats import zonal_stats
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
import rasterio
import json
import pandas as pd
from osgeo import gdal, ogr, osr

root = tk.Tk()
root.withdraw()
root.attributes("-topmost", True)

''

## Clip raster

In [6]:
def ClipRasterByExtent(raster, admin, workspace):
    bbox = admin.total_bounds
    bbox2 = [bbox[0], bbox[3], bbox[2], bbox[1]]
    clipped = gdal.Translate('', raster, format = 'MEM', projWin = bbox2, noData = 0)
    return clipped

## Reclassify raster

In [7]:
def ReclassifyRasters(file):    
    driver = gdal.GetDriverByName("MEM")
    band = file.GetRasterBand(1)
    lista = band.ReadAsArray()
    
    lista[np.where((0 < lista) & (lista < 999999999))] = 1

    file2 = driver.Create('', file.RasterXSize , file.RasterYSize , 1, gdal.GDT_Float32)
    file2.GetRasterBand(1).WriteArray(lista)

    proj = file.GetProjection()
    georef = file.GetGeoTransform()
    file2.SetProjection(proj)
    file2.SetGeoTransform(georef)
    file2.FlushCache()
    
    return file2

## Resample raster

In [8]:
def ResampleRaster(raster, factor):
    gt =raster.GetGeoTransform()
    xRes = factor*gt[1]
    yRes = factor*gt[1]
    kwargs1 = {'noData':'0'}
    resamp1 = gdal.Translate('',raster, format ='MEM', **kwargs1)
    kwargs2 = {'xRes': xRes, 'yRes': yRes, 'resampleAlg':'mode'}    
    resampled = gdal.Translate('',resamp1, format ='MEM', noData = 0, **kwargs2)
    
    return resampled

## Rasterize Polygon

In [9]:
def rasterize(vector, vector_path, raster, output):
    
    vector["id"] = np.arange(len(vector))+1
    vector.to_file(vector_path)
    data = raster

    geo_transform = data.GetGeoTransform()
    x_min = geo_transform[0]
    y_max = geo_transform[3]
    x_max = x_min + geo_transform[1] * data.RasterXSize
    y_min = y_max + geo_transform[5] * data.RasterYSize
    x_res = data.RasterXSize
    y_res = data.RasterYSize
    mb_v = ogr.Open(vector_path)
    mb_l = mb_v.GetLayer()
    pixel_width = geo_transform[1]
    target_ds = gdal.GetDriverByName('GTiff').Create(output, x_res, y_res, 1, gdal.GDT_Byte)
    target_ds.SetGeoTransform((x_min, pixel_width, 0, y_max, 0, -pixel_width))
    target_dsSRS = osr.SpatialReference()
    target_dsSRS.ImportFromEPSG(4326)
    target_ds.SetProjection(target_dsSRS.ExportToWkt())
    band = target_ds.GetRasterBand(1)
    NoData_value = -999999
    band.SetNoDataValue(NoData_value)
    band.FlushCache()
    gdal.RasterizeLayer(target_ds, [1], mb_l, options=["ATTRIBUTE=id"])
    target_ds = None

## Rastercalculator

In [10]:
def array_calc(rstpth1,rstpth2, output, filetype = gdal.GDT_Float32):
    
    rst1 = gdal.Open(rstpth1)
    band_data1 = rst1.GetRasterBand(1)
    a = band_data1.ReadAsArray()

    rst2 = gdal.Open(rstpth2)
    band_data2 = rst2.GetRasterBand(1)
    b = band_data2.ReadAsArray()
    
    f = b * a
        
    ref = gdal.Open(rstpth1)
    band = ref.GetRasterBand(1)
    proj = ref.GetProjection()
    geotransform = ref.GetGeoTransform()
    xsize = band.XSize
    ysize = band.YSize
    
    driver = gdal.GetDriverByName('GTiff') 
    out = driver.Create(output, xsize, ysize, 1, filetype)
    out.GetRasterBand(1).WriteArray(f) 
    
    out.SetProjection(proj)
    out.SetGeoTransform(geotransform)
    out.FlushCache()
    out = None
    x = gdal.Open(output)
    return x

## Urban/rural split

In [11]:
def CalibrateUrban(clusters, urban_current, workspace):      
    
    urban_modelled = 2
    factor = 1
    pop_tot = clusters["Popsum"].sum()
    
    while abs(urban_modelled - urban_current) > 0.001:
        clusters["IsUrban"] = 0
        
        clusters.loc[(clusters["Popsum"] > 5000 * factor) & (
            clusters["Popsum"] / clusters["GridCellAr"] > 350 * factor), "IsUrban"] = 1
        clusters.loc[(clusters["Popsum"] > 50000 * factor) & (
            clusters["Popsum"] / clusters["GridCellAr"] > 1500 * factor), "IsUrban"] = 2
        pop_urb = clusters.loc[clusters["IsUrban"] > 1, "Popsum"].sum()
        
        urban_modelled = pop_urb / pop_tot
        
        if urban_modelled > urban_current:
            factor *= 1.1
        else:
            factor *= 0.9
            
    clusters.to_file(workspace + r"/clusters.shp") 
    
    print("modelled urban ratio is " + str(urban_modelled) + "% in comparision to the actual ratio of " + str(urban_current) + "%")


## Convert raster to polygon

In [12]:
def ToPolygon(Raster, opt, output):
    
    if type(Raster) == str:
        Raster = gdal.Open(Raster)
    
    band = Raster.GetRasterBand(1)
    bandArray = band.ReadAsArray()
    
    outShapefile = output
    
    driver = ogr.GetDriverByName("ESRI Shapefile")
    if os.path.exists(outShapefile+".shp"):
        driver.DeleteDataSource(outShapefile+".shp")
    outDatasource = driver.CreateDataSource(outShapefile+ ".shp")
    
    outLayer = outDatasource.CreateLayer(outShapefile+ ".shp", srs=None)
    newField = ogr.FieldDefn('PLACEHOLDE', ogr.OFTInteger)
    outLayer.CreateField(newField)
    gdal.Polygonize(band, band, outLayer, 0, ["8CONNECTED=8","GROUPBY=PLACEHOLDE"], callback=None)
    outDatasource.Destroy()
    sourceRaster = None
    if opt ==1:
        out = gpd.read_file(outShapefile+".shp")
    else:
        NTLArea=gpd.read_file(outShapefile+".shp")
        clean = NTLArea[NTLArea.PLACEHOLDE != 0]
        clean_b = clean.buffer(0)
        clean_b.crs = {'init' :'epsg:4326'}
        out = clean_b
        out = clean_b
    
    return out      

## Clip raster by mask and save

In [13]:
def ClipRasterByMask(raster_path, mask_path, crs, output):
    with fiona.open(mask_path, "r") as shapefile:
        shapes = [feature["geometry"] for feature in shapefile]
    with rasterio.open(raster_path) as src:
        out_image, out_transform = rasterio.mask.mask(src, shapes, crop=True)
        out_image[out_image<0] = np.nan
        mask = (out_image!=0)
        out_image = fillnodata(out_image, mask)
        out_meta = src.meta
    out_meta.update({"driver": "GTiff",
                     "height": out_image.shape[1],
                     "width": out_image.shape[2],
                     "transform": out_transform,
                     "crs": crs})
    
    out_meta.update(compress = 'lzw')
    
    with rasterio.open(output, "w", **out_meta) as dest:
        dest.write(out_image)
    out = rasterio.open(output)
        
    return out

## Save memory raster


In [14]:
def saveRaster(input_file, output_file):
    kwargs = {'creationOptions': ['COMPRESS=LZW']}
    gdal.Warp(output_file, input_file, **kwargs)

## Splitting and populating clusters

In [15]:
def addAttributes(clusters, crs, country_name):
    clusters['id'] = np.arange(len(clusters))
    clusters.crs = {'init' :'epsg:4326'}
    clusters_proj = clusters.to_crs({ 'init': crs})
    clusters_proj["GridCellAr"] = clusters_proj.area/1000000
    clusters_proj["Country"] = country_name
    clusters = clusters_proj.to_crs({ 'init': 'epsg:4326'})
    clusters = clusters.drop(['PLACEHOLDE'], axis=1)
    return clusters

## Populating clusters

In [16]:
def populatingClusters(clusters,raster,column,method):
    clusters = zonal_stats(
    clusters,
    raster.name,
    stats=[method],
    prefix=column, geojson_out=True, all_touched=True)
    
    return clusters

## Finalizing clusters

In [17]:
def finalizing_clusters(clusters, workspace):
    output = workspace + r'\placeholder.geojson'
    with open(output, "w") as dst:
        collection = {
            "type": "FeatureCollection",
            "features": list(clusters)}
        dst.write(json.dumps(collection))
  
    clusters = gpd.read_file(output)
    clusters.fillna(0, inplace=True)
    os.remove(output)
    
    clusters = clusters.rename(columns={"popsum": "Pop"})
    clusters = clusters.rename(columns={"NTLmax": "NightLight"})
    clusters = clusters.rename(columns={"area": "Area"})
    clusters = clusters.rename(columns={"ElecPopsum": "ElecPop"})
    
    clusters.to_file(workspace + r"\clusters.shp")

    dir_name = workspace
    test = os.listdir(dir_name)

    for item in test:
        if item.endswith(".tif") or item.startswith("NTLArea") or item.startswith("bufferedlines"):
            os.remove(os.path.join(dir_name, item))
            
    return clusters